In [15]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

pd.set_option("display.max_rows", None, "display.max_columns", None) #Çıktı satırlarının tümünün gösterilmesini sağlar

np.set_printoptions(threshold=np.inf)

In [2]:
def df_list (index):
    df_list = []
    for x in index:
        df_keep = df[df.index == x ]
        df_list.extend(df_keep)
    return df_list

In [3]:
#User ID listesini İndex listesine dönüştürmek için forksiyon
def index_list (user_id):
    index_list = []
    for user in user_id:
        index_keep = df.index[df.user == user ]
        index_list.extend(index_keep)
    return index_list

In [4]:
df = pd.read_json (r'..\SteamData\SteamGameTime_clear.json')

In [5]:
df = df.drop("playtime_2weeks" , axis = 1) #playtime_2weeks sütununu sildim

In [6]:
df.head()

,appid,playtime_forever,user
0,620,76,76561198258109920
1,261550,2885,76561198258109920
2,255710,1061,76561198258109920
3,381210,498,76561198258109920
4,289070,211,76561198258109920


In [7]:
zero_playtime_id = df.index[df.playtime_forever == 0]

In [8]:
len(zero_playtime_id)

744538

In [9]:
#Liste indexlerine göre satırların silinmesi 
df_checkpoints = df.copy()
df.drop(zero_playtime_id, axis = 0, inplace = True)

In [10]:
print("*Satır sayısı checkpoints = ",len(df_checkpoints))
print("*Satır sayısı = ",len(df))

*Satır sayısı checkpoints =  1765418
*Satır sayısı =  1020880


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996974 entries, 0 to 3877974
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   appid             996974 non-null  int64
 1   playtime_forever  996974 non-null  int64
 2   user              996974 non-null  int64
dtypes: int64(3)
memory usage: 30.4 MB


In [2]:
#df = pd.read_json (r'..\SteamData\SteamGameTime_GameClear.json')

In [13]:
#18 Kişiden az sayıda oyuncusu olan oyunların çıkartılması
appid_value = df["appid"].value_counts()
df_checkpoints = df.copy()

for i in appid_value.index:
    if (appid_value[i] < 10): 
        index_keep = df.index[df.appid == i]
        df.drop(index_keep, axis = 0, inplace = True)

In [14]:
print("*Oyun sayısı checkpoints = ",len(df_checkpoints.appid.unique()))
print("*Oyun sayısı = ",len(df.appid.unique()))
print(" Eksilen oyun sayısı = ",len(df_checkpoints.appid.unique()) - len(df.appid.unique()))

*Oyun sayısı checkpoints =  13214
*Oyun sayısı =  4832
 Eksilen oyun sayısı =  8382


In [15]:
df.to_json(r'..\SteamData\SteamGameTime_GameClear.json', index = bool )

----------------------------------------------------------------------------------------------

In [16]:
user_statistics = pd.read_json (r'..\SteamData\SteamUser_statistics.json')

In [17]:
user_statistics["play_len"] = user_statistics["played_len"] + user_statistics["zero_len"]

In [18]:
len(user_statistics.user_id[user_statistics.played_len < 10])

1855

In [24]:
df_index = index_list(user_statistics.user_id[user_statistics.played_len < 10])

In [25]:
df.drop(df_index, axis = 0, inplace = True)

--------------------------------------------------------------------------------------------------

Tüm oyunlar varken:

    <17825x13214 sparse matrix of type '<class 'numpy.intc'>'
	with 1019790 stored elements in Compressed Sparse Row format>
    
    99.5670408642625
    
Oyunlar n>10 olduğunda:

    <17825x4625 sparse matrix of type '<class 'numpy.intc'>'
	with 993817 stored elements in Compressed Sparse Row format>
    
    98.79450574276942
    
Oyunlar n>18 olduğunda (0,001):

    <17825x3551 sparse matrix of type '<class 'numpy.intc'>'
	with 978682 stored elements in Compressed Sparse Row format>
    
    98.45381523407862
    
Oyunlar n>18 olduğunda (0,001), kişiler oyun sayısı 10'dan küçük olanlar:

    <15769x3551 sparse matrix of type '<class 'numpy.intc'>'
	with 966876 stored elements in Compressed Sparse Row format>
    
    98.27330371452146
    
Oyunlar n=>10 olduğunda (0,001), kişiler oyun sayısı 10'dan küçük olanlar:

    <15769x4832 sparse matrix of type '<class 'numpy.intc'>'
	with 984009 stored elements in Compressed Sparse Row format>
    
    98.70857856117229

In [4]:
users = list(np.sort(df.user.unique())) #unique kulanıcı listesi sıralı
appids = list(df.appid.unique()) #unique appid listesi
playtime_forever = list(df.playtime_forever)
#astype('category') veriyi katagorik yapıyor #.cat.codes veriye katagori veriyor #örnek 76561198258109920 = 8606 oluyor
rows = df.user.astype('category').cat.codes 
cols = df.appid.astype('category').cat.codes 
playtime_sparse = sparse.csr_matrix((playtime_forever, (rows, cols)), shape=(len(users), len(appids)))

In [5]:
playtime_sparse

<17825x4832 sparse matrix of type '<class 'numpy.intc'>'
	with 995887 stored elements in Compressed Sparse Row format>

In [7]:
matrix_size = playtime_sparse.shape[0]*playtime_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(playtime_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.84374506562142

In [11]:
x = playtime_sparse.toarray()

In [16]:
x

KeyboardInterrupt: 